# Info
Phase Shift Full Bridge Converter Design tool for battery charger.
Design for variable output voltage with limited maximum output current and power.

# QSPICE Python Files Initialization

In [40]:
import os
import shutil

# Get the current working directory (where your script is running)
current_dir = os.getcwd()

# Define the source folder (QSPICE_Parser is located above the project directory)
source_folder = os.path.join(os.path.dirname(current_dir), 'QSPICE_Parser')

# Define the source file path
source_file = os.path.join(source_folder, 'QSPICE_parser.py')

# Define the destination folder within the project (subfolder called QSPICE)
destination_folder = os.path.join(current_dir, 'QSPICE')

# Define the destination file path (within the QSPICE subfolder)
destination_file = os.path.join(destination_folder, 'QSPICE_parser.py')

# Copy the file from source to destination
shutil.copy(source_file, destination_file)

'c:\\Users\\Stani\\OneDrive\\JupyterLab\\GitHub\\PowerElectronics_JupyterLab_QSPICE\\Phase-Shift_Full-Bridge\\QSPICE\\QSPICE_parser.py'

In [41]:
from QSPICE import QSPICE_parser
import importlib
import sys

# reload QSPICE/Python parser without kernel restart
importlib.reload(QSPICE_parser)

#Create PyQSPICE file
QSPICE_parser.parse_and_generate_script('PSFB_center_tap.qsch','QSPICE_PSFB_center_tap.py')

# Delete parser script file from QSPICE folder
os.remove(destination_file)

In [42]:
import math
import numpy as np
import pandas as pd
import sympy as sp
import math

#from IPython.display import display, clear_output
#from IPython import display
from IPython.display import display, update_display
import time

%matplotlib ipympl
#load all QSPICE autogenerating scripts here
from QSPICE import QSPICE_PSFB_center_tap as QSPICE

from IPython.display import display, Markdown, Latex
from matplotlib import pyplot as plt
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from scipy.interpolate import RegularGridInterpolator
from scipy.interpolate import interp1d

plt.close('all')
pd.options.display.float_format = '{:.5e}'.format

def get_Lout(N, f__s,V__d, I__L_pp,V__input,Vout__min,Vout__max):

    """
    Calculates minimum output inductor value which will ensure that I_Lout_pp < dIL_pp in whole Vout range

    Parameters:
        N (float): Transformer turn ratio (Np/Ns).
        f__s (float): Primary switching frequency.
        V__d (float): Output diode voltage drop.
        I__L_pp (float): Output inductor peak-to-peak ripple current.
        V__input (float): Input voltage.
        Vout__min (float): Minimum output voltage.
        Vout__max (float): Maximum output voltage.

    Returns:
        (Vout, Lout): Output voltage when inductor current ripple is maximal / Required minimal inductance.

    """
    x = sp.Symbol('x')
    equation = x / f__s / I__L_pp * (1 - x / (V__input / N - V__d)) / 2
    derivative = sp.diff(equation,x)
    critical_points = sp.solvers.solve(derivative, x)
    interval = (Vout__min,Vout__max)
    points_to_check = [p for p in critical_points if interval[0] <= p <= interval[1]]+list(interval)
    values = [(point,equation.subs(x,point)) for point in points_to_check]
    Vout, Lout = max(values, key=lambda item: item[1])
    
    return Vout, Lout


def find_OP(Coss, Cout_ESR, D, deadtime, eff, fsw_p, Iout_max, L_leak, Lout, Lp, Lr, N, Pmax, Rdiff, REC_fwd, Ron_MOS, Vin, Vout):
    """
    Uses the bisection method to find duty-cycle D such that the relative
    difference between I_Lout_AVG and I_Rload_AVG is below 0.1%.

    It returns D.

    """
    #possible duty-cycle interval to find OP
    D_min=0.01
    D_max=0.99

    #acceptable error tolerance
    tolerance=0.001
    #maximum number iterations
    max_iterations=20

    def relative_difference(I_Lout_AVG, I_Rload_AVG):
        return abs(I_Lout_AVG - I_Rload_AVG) / ((I_Lout_AVG + I_Rload_AVG) / 2)
    
    display("Starting iterations...", display_id='1')

    for iteration in range(max_iterations):
        # Calculate midpoint of D range
        D_mid = (D_min + D_max) / 2
        
        # Simulate with the midpoint value
        results_sim = QSPICE.QSPICE_PSFB_center_tap(Coss, Cout_ESR, D_mid, deadtime, eff, fsw_p, Iout_max, L_leak, Lout, Lp, Lr, N, Pmax, Rdiff, REC_fwd, Ron_MOS, Vin, Vout)
        I_Lout_AVG = results_sim[1][0]
        I_Rload_AVG = results_sim[1][1]
        
        # Calculate the relative difference
        diff = relative_difference(I_Lout_AVG, I_Rload_AVG)

        update_display(f"Iteration: {iteration + 1}, D={D_mid:.4g}, Relative difference={diff:.4g}", display_id='1')

        # Check if the difference is within the tolerance
        if diff <= tolerance:
            print(f"Converged after {iteration + 1} iterations.")
            return D_mid, results_sim

        # Adjust the bounds based on the simulation result
        if I_Lout_AVG > I_Rload_AVG:
            D_max = D_mid  # Narrow the range to the lower half
        else:
            D_min = D_mid  # Narrow the range to the upper half

        
        
    # If max_iterations is reached without convergence
    raise RuntimeError(
        f"Failed to converge within {max_iterations} iterations. "
        f"Last D: {D_mid}, I_Lout_AVG: {I_Lout_AVG}, I_Rload_AVG: {I_Rload_AVG}"
    )

# Input Data

In [43]:
#Input DC range
Vin_min=380
Vin_max=420
Vin_nom=400

#Output voltage range
Vout_min=172 
#Vout_nom=264.7
Vout_max=300
Vout_pp=50e-3 # Peak-to-peak Vout ripple at resistive load 

#Output power
Pout_max=900

#Output current limit
Iout_max=3.4

#Estimated efficiency
eff=0.95

#switching frequency (on the primary side, seen by switching transistor)
#note: the output inductor is switched in 2*fsw
fsw=100e3

#maximum effective secondary duty-cycle
#Note: Consider duty-cycle loss due to resonant inductor + transformer primary leakage inductance
DC_eff_max=0.85

#PWM IC duty cycle limit
DC_PWM_lim=0.95

#### Output Inductor ####
dIL_pp=1.53 #inductor peak to peak ripple current (in A)
L_out_rdc = 10e-3 #inductor rdc

#### Resonant Inductor ####
Lr_rdc = 10e-3 #inductor rdc

#### Transformer ####
#Estimation of transformer parasitic parameters
#Primary winding capacitance
Cp = 100e-12
#Primary leakage inductance
L_leak = 8e-6

Vsec_k=1.24

#### MOSFET Parameters ####
# VDS derating
kM=0.85
# WC rdson at 110degC
M_rdson=0.14
# Energy related Coss in F at Vin_max
Coss_ef = 85e-12

#### Output Diodes Parameters@75degC ####
# MFR / MPN

kD=0.6 # Vbr derating
D_Vfwd=0.69 # Build-in potential (in V)
D_rdiff=76e-3 # Differential resistance (in Ohm)

### Output Diode@75degC ###
# Infineon IDH08G65C6


# Transformer Requirements

## Turns Ratio (Np:1)

In [44]:
Vsec_min=(Vout_max+D_Vfwd)/DC_eff_max
N=round(Vin_min/Vsec_min,2)
print(N)

1.07


# Resonant Inductor

## Maximum Duty-Cycle loss (in %)

In [45]:
DC_loss_max=DC_PWM_lim-DC_eff_max
print(round(DC_loss_max*100,2))

10.0


## Estimated Maximum Inductance of Resonant Inductor (in uH)

In [46]:
Lr_max = DC_loss_max*N*Vin_min/((Pout_max/Vout_max)*4*fsw)-L_leak
print(round(Lr_max*1e6,2))

25.88


# Output Inductor Requirements
Output inductor is calculated such as Iripple < Iripple_limit at whole Vout range. This happen when Vin is maximal

## Required Inductance (in uH)

In [47]:
Lout = round(get_Lout(N, fsw, D_Vfwd, dIL_pp, Vin_max, Vout_min, Vout_max)[1],5)
Vout_WC_I_ripple = round(get_Lout(N, fsw, D_Vfwd, dIL_pp, Vin_max, Vout_min, Vout_max)[0],5)
#Lout = (Pout_max/Iout_max) / fsw / dIL_pp * (1 - (Pout_max/Iout_max) / Vin_max * N) / 2
print(int(Lout*1e6))
print(Vout_WC_I_ripple)


319
195.91668


# Output Capacitor Requirements

## Minimum Capacitance Value (in uF)
Note: This assumes that output capacitor ESR is 0 Ohm

In [48]:
Cout = Vout_WC_I_ripple / Lout / fsw ** 2 / Vout_pp * (1 - Vout_WC_I_ripple / (Vin_max / N - D_Vfwd)) / 32
print(round(Cout*1e6))

19


## Maximum ESR limit (in mOhm)
Note: This assumes that output capacitance is infinity

In [49]:
Cout_ESR_max = Vout_pp/dIL_pp
print(round(Cout_ESR_max*1000))

33


## Estimated Electrolytical Capacitor (in uF)
Note: Based on assumption that relationship between the capacitance and ESR of an
electrolytic capacitor is C ⋅ ESR = 60e-6

In [50]:
Cout_elyt = 60e-6/Cout_ESR_max
print(round(Cout_elyt*1e6))

1836


# ZVS Range

## Required energy to achieve ZVS (in uJ)

In [51]:
E_TR = 0.5* Cp*Vin_max*Vin_max
E_Coss = (0.5*Coss_ef*Vin_max*Vin_max)*2
E_ZVS=E_TR+E_Coss
print(round(E_ZVS*1e6,2))

23.81


In [52]:
test = find_OP(30e-12, Cout_ESR_max, 0 , 50e-9,1,fsw,Iout_max,6e-13,Lout, 4e-3,Lr_max,N,Pout_max,30e-3,0.9,180e-3,Vin_max,Vout_WC_I_ripple)
print(test[0])
sim_results = test[1]
print("I_Lout_AVG:",sim_results[1][0])
print("Rload_AVG:",sim_results[1][1])
print("I_S1_RMS:",sim_results[1][5])
print("I_Lout_PP:",sim_results[1][6])

'Iteration: 13, D=0.576, Relative difference=0.0004207'

Converged after 13 iterations.
0.5759643554687501
I_Lout_AVG: 3.39857
Rload_AVG: 3.4
I_S1_RMS: 2.18924
I_Lout_PP: 1.4305
